# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_api_df = pd.read_csv("Output/weather.csv")
weather_api_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Punta Arenas,CL,-53.15,-70.92,1594685099,90,80,35.6,8.05
1,Nieuw Amsterdam,SR,5.88,-55.08,1594685100,40,83,80.6,8.05
2,Bredasdorp,ZA,-34.53,20.04,1594685101,70,87,46.4,13.87
3,Cape Town,ZA,-33.93,18.42,1594685101,75,76,52.0,8.05
4,Barsovo,RU,61.17,73.17,1594685101,1,87,57.2,8.95


In [3]:
weather_api_df.dtypes

City           object
Country        object
Latitude      float64
Longitude     float64
Date            int64
Cloudiness      int64
Humidity        int64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = weather_api_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_api_df["Humidity"].astype(float)
max_humidity = humidity.max()
locations.head()

,Latitude,Longitude
0,-53.15,-70.92
1,5.88,-55.08
2,-34.53,20.04
3,-33.93,18.42
4,61.17,73.17


In [6]:
fig = gmaps.figure(center=(50, -5.0), zoom_level=4)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
plt.savefig("VacationHeatMap.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
perfect_weather_df = weather_api_df.loc[(weather_api_df["Max Temp"] > 70) & (weather_api_df["Max Temp"] < 80) & (weather_api_df["Cloudiness"] ==0), :]
perfect_weather_df = perfect_weather_df.dropna(how="any")
perfect_weather_df.reset_index(inplace=True)
del perfect_weather_df["index"]
perfect_weather_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Akdepe,TM,42.06,59.38,1594685108,0,38,77.00,6.93
1,Arraial do Cabo,BR,-22.97,-42.03,1594685110,0,72,73.36,2.59
2,Piaçabuçu,BR,-10.41,-36.43,1594685140,0,86,74.16,5.99
3,Lata,PT,40.16,-8.33,1594685145,0,73,71.01,1.01
4,Kumul,CN,42.80,93.45,1594685149,0,30,74.75,8.08


In [8]:
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,Akdepe,TM,42.06,59.38,1594685108,0,38,77.00,6.93,
1,Arraial do Cabo,BR,-22.97,-42.03,1594685110,0,72,73.36,2.59,
2,Piaçabuçu,BR,-10.41,-36.43,1594685140,0,86,74.16,5.99,
3,Lata,PT,40.16,-8.33,1594685145,0,73,71.01,1.01,
4,Kumul,CN,42.80,93.45,1594685149,0,30,74.75,8.08,


In [9]:
hotel_list =[]

for i in range(len(perfect_weather_df)):
    latitude = perfect_weather_df.loc[i]["Latitude"]
    longitude = perfect_weather_df.loc[i]["Longitude"]
    
    hotel_location = f"{latitude},{longitude}"
    
    params= {
        "location": hotel_location,
        "radius": 5000,
        "types" : "hotel",
        "key": g_key 
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    
    response = requests.get(url, params=params).json()
    
    try:
        hotel_list.append(response["results"][0]["name"])

    except:
        hotel_list.append(f"")
perfect_weather_df["Hotel Name"] = hotel_list
perfect_weather_df = perfect_weather_df.dropna(how="any")
perfect_weather_df.head()
 

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,Akdepe,TM,42.06,59.38,1594685108,0,38,77.00,6.93,Akdepe
1,Arraial do Cabo,BR,-22.97,-42.03,1594685110,0,72,73.36,2.59,Cabo Frio
2,Piaçabuçu,BR,-10.41,-36.43,1594685140,0,86,74.16,5.99,Piaçabuçu
3,Lata,PT,40.16,-8.33,1594685145,0,73,71.01,1.01,Coimbra
4,Kumul,CN,42.80,93.45,1594685149,0,30,74.75,8.08,Hami


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_weather_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))